单调队列是一种严格单调的队列，可以单调递增，也可以单调递减。队首位置保存的是最优解，第二个位置保存的是次优解，ect...


单调队列可以有两个操作：


1、插入一个新的元素，该元素从队尾开始向队首进行搜索，找到合适的位置插入之，如果该位置原本有元素，则替换它。


2、在过程中从队首删除不符合当前要求的元素。

 

单调队列实现起来可简单，可复杂。简单的一个数组，一个head，一个tail指针就搞定。复杂的用双向链表实现。

 

用处：


1、保存最优解，次优解，ect...


2、利用单调队列对dp方程进行优化，可将O(n)复杂度降至O(1)。也就是说，将原本会超时的N维dp降优化至N-1维，以求通过。这也是我想记录的重点


是不是任何DP都可以利用单调队列进行优化呢？答案是否定的。


只有形如 dp[i]=max/min (f[k]) + g[i]  （k<i && g[i]是与k无关的变量）才能用到单调队列进行优化。


优化的对象就是f[k]。

例一：

给你一个长度为n的整数序列，要求从中找出一段连续的长度不超过m的子序列，使得这个序列的和最大

In [4]:
def max_subarray(nums, m):
    if not nums or m < 1:
        return 0
    n = len(nums)
    s = [0]*(n+1)
    for i in range(n):
        s[i+1] = s[i] + nums[i]
    head, tail = 0, 0
    result = float('-inf')
    temp = [(0, 0)]*(n+1)
    for i in range(1, n+1):
        while head <= tail and i - temp[head][0] > m:
            head += 1 
        result = max(result, s[i] - temp[head][1])
        while head <= tail and temp[tail][1] > s[i]:
            tail -= 1
        tail += 1
        temp[tail] = (i, s[i])
        
    return result
                    
nums = [1, -3, 5, 1, -2, 3]
m = 4
print(max_subarray(nums, m))

7


例二:
滑动窗口：给一个长度为n的数组，一个长为k的滑动窗体从最左端移至最右端，你只能看到窗口中的 k个数，每次窗体向右移动一位,找出窗体在各个位置时的最大值和最小值

In [21]:
def sliding_window_minmax(nums, k):
    n = len(nums)
    if n < k:
        return [0, 0]
    result = [[0]*(n-k+1), [0]*(n-k+1)]
    min_array, max_array = [(1, nums[0])]*(n-k+1), [(1, nums[0])]*(n-k+1)
    min_head, min_tail, max_head, max_tail = 0, 0, 0, 0
    for i in range(1, n):
        while min_head <= min_tail and min_array[min_tail][1] > nums[i]:
            min_tail -= 1
        min_tail += 1
        min_array[min_tail] = (i+1, nums[i])
        while min_head <= min_tail and i+1 - min_array[min_head][0] >= k:
            min_head += 1 
        while max_head <= max_tail and max_array[max_tail][1] < nums[i]:
            max_tail -= 1
        max_tail += 1
        max_array[max_tail] = (i+1, nums[i])
        while max_head <= max_tail and i+1 - max_array[max_head][0] >= k:
            max_head += 1
            
        if i >= k-1:
            result[0][i-k+1] = min_array[min_head][1]
            result[1][i-k+1] = max_array[max_head][1]
    return result

k = 3
nums = [1, 3, -1, -3, 5, 3, 6, 7]
result = sliding_window_minmax(nums, k)
print(result[0])
print(result[1])


[-1, -3, -3, -3, 3, 3]
[3, 3, 5, 5, 6, 7]


例三： 单调队列来优化多重背包问题

In [22]:
def multiplebagpack(weight, value, nums, capacity):
    n = len(weight)
    dp = [0]*(capacity+1)
    for i in range(n):
        nums[i] = min(nums[i], capacity//weight[i])
    for i in range(n):
        for mod in range(weight[i]):
            temp = [0]*(max(nums[i], capacity//weight[i])+1)
            head, tail = 0, 0            
            for k in range(1, (capacity-mod)//weight[i]+1):
                temp[0] = (0, dp[mod])
                while head <= tail and dp[k*weight[i]+mod] - k*value[i] > temp[tail][1]:
                    tail -= 1
                tail += 1
                temp[tail] = (k, dp[k*weight[i]+mod] - k*value[i])
                while head <= tail and temp[head][0] < k - nums[i]:
                    head += 1                    
                dp[k*weight[i]+mod] = temp[head][1] + k*value[i]
    return dp[-1]

weight = [3, 4, 5, 6, 7]
value = [5, 6, 9, 10, 11]
nums = [1, 5, 1, 5, 5]
capacity = 13
print(multiplebagpack(weight, value, nums, capacity))

21
